In [ ]:
!python train_classifier_ssfl.py --data_name voc --model_name deeplab --control_name 500_fix@0.95-mix_3_0.5_iid_5-5_0.5_0_1

In [ ]:
git clone https://github.com/Karthik2924/SemiFL_extension.git

In [3]:
mkdir data

In [4]:
cd data

/home/ubuntu/SemiFL_extension/src/data


In [6]:
!gdown 1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65

Downloading...
From (original): https://drive.google.com/uc?id=1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65
From (redirected): https://drive.google.com/uc?id=1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65&confirm=t&uuid=952b0318-c80b-4622-8d8d-5bd4d1153b50
To: /home/ubuntu/SemiFL_extension/src/data/images.npy
100%|███████████████████████████████████████| 2.29G/2.29G [00:14<00:00, 154MB/s]


In [7]:
!gdown 1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ

Downloading...
From (original): https://drive.google.com/uc?id=1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ
From (redirected): https://drive.google.com/uc?id=1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ&confirm=t&uuid=4c3f81bc-da88-4f68-97e0-0dbb8c172905
To: /home/ubuntu/SemiFL_extension/src/data/masks.npy
100%|███████████████████████████████████████| 1.53G/1.53G [00:09<00:00, 165MB/s]


In [2]:
pip install gdown torchmetrics anytree

[Errno 2] No such file or directory: 'data'
/home/ubuntu/SemiFL_extension/src


In [18]:
from itertools import compress
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

In [19]:
import torch
from torch.utils.data import Dataset

class SimpleDataset(Dataset):
    def __init__(self):
        #self.num_samples = num_samples
        self.data = torch.rand(100,3,128,128)
        self.target = torch.rand(100,1,128,128)
        self.transform = None
        self.ind = np.arange(100)


    def __len__(self):
        return len(self.data)
        #return self.num_samples

    def __getitem__(self, idx):
        didx = self.ind[idx]
        if self.transform:
            return {'data':self.transform(torch.tensor(self.data[didx])), 'target':self.transform(self.target[idx])}
        else:
            return {'data': torch.tensor(self.data[didx]), 'target':self.target[idx]}

# Create an instance of the ToyDataset
toy_dataset = SimpleDataset()


In [ ]:
 mix_data : torch.Size([32, 3, 256, 256])
 mix_target : torch.Size([32, 256, 256, 2])

In [55]:
mt = torch.rand(32,256,256,2)
mt[...,0].shape

torch.Size([32, 256, 256])

In [225]:
toy_dataset[9]

{'data': tensor([[[0.1558, 0.5554, 0.7492,  ..., 0.7182, 0.6796, 0.6616],
          [0.0715, 0.1611, 0.1820,  ..., 0.7437, 0.8855, 0.1173],
          [0.9753, 0.6165, 0.5458,  ..., 0.2451, 0.8714, 0.9966],
          ...,
          [0.7623, 0.5801, 0.5785,  ..., 0.9306, 0.1995, 0.6560],
          [0.3178, 0.5615, 0.6404,  ..., 0.4690, 0.6695, 0.1623],
          [0.7191, 0.6891, 0.3419,  ..., 0.6456, 0.5513, 0.0148]],
 
         [[0.7347, 0.0358, 0.0323,  ..., 0.3718, 0.2759, 0.4531],
          [0.5653, 0.4994, 0.9425,  ..., 0.9436, 0.1095, 0.9461],
          [0.7066, 0.5367, 0.4919,  ..., 0.4237, 0.4636, 0.9858],
          ...,
          [0.5698, 0.4981, 0.6093,  ..., 0.7908, 0.8211, 0.4857],
          [0.8211, 0.7413, 0.0222,  ..., 0.0151, 0.6716, 0.4168],
          [0.1369, 0.1983, 0.8129,  ..., 0.9479, 0.2614, 0.0351]],
 
         [[0.6801, 0.1116, 0.7881,  ..., 0.2024, 0.5684, 0.4633],
          [0.6290, 0.7158, 0.5946,  ..., 0.4991, 0.0343, 0.2133],
          [0.4969, 0.4387, 0.126

In [ ]:
output_, input_ = {}, {}
output_['target'] = torch.cat(output, dim=0)
input_['target'] = torch.cat(target, dim=0)
#output_['target'] = F.softmax(output_['target'], dim=-1)
output_['target'] = F.softmax(output_['target'], dim=1)

In [3]:
op = torch.rand(12, 21, 256, 256)
ip = torch.randint(0,21,(12,256,256))
print(op.shape,ip.shape)

torch.Size([12, 21, 256, 256]) torch.Size([12, 256, 256])


In [4]:
op = F.softmax(op, dim=1)

In [5]:
op.shape

torch.Size([12, 21, 256, 256])

In [6]:
def make_hard_pseudo_label( soft_pseudo_label):
    #max_p, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1)
    max_p, hard_pseudo_label = torch.max(soft_pseudo_label, dim=1)
    print(max_p.shape)
    mv = max_p.view(max_p.shape[0],-1).sum(1)
    a,b,c = max_p.shape
    total = b*c
    #mask = max_p.ge(cfg['threshold'])
    #mask = max_p.ge(0.08)
    mask = mv.ge(total*0.07225)
    return hard_pseudo_label, mask


In [7]:
nlab,mask = make_hard_pseudo_label(op)

torch.Size([12, 256, 256])


In [8]:
torch.unique(mask,return_counts = True)

(tensor([False,  True]), tensor([7, 5]))

In [210]:
sup = list(compress(nlab, mask))

In [226]:
nlab.shape

torch.Size([12, 256, 256])

In [9]:
fdata = torch.rand(25,3,256,256)

In [10]:
ind = torch.arange(12)

In [11]:
fdata[ind,:][mask].shape

torch.Size([5, 3, 256, 256])

In [12]:
r = torch.rand(12,256,256)
r.view(12,-1).sum(1).shape

torch.Size([12])

In [246]:
mask

tensor([False, False, False,  True, False,  True,  True, False, False, False,
        False,  True])

In [228]:
torch.stack(sup).shape

torch.Size([4, 256, 256])

In [45]:
sup1 = list(compress(ip, mask))

In [48]:
type(torch.stack(sup1)) == torch.Tensor

True

In [47]:
type(sup1)

list

In [231]:
ip.shape

torch.Size([12, 256, 256])

In [13]:
from torch.utils.data import DataLoader, Dataset
dl = DataLoader(toy_dataset,batch_size = 16,shuffle = False)

In [239]:
batch = next(iter(dl))

In [241]:
b1 = next(iter(dl))

In [243]:
b2 = next(iter(dl))

In [245]:
b1['data'] == batch['data'], b2['data'] == batch['data']

(tensor([[[[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]],
 
          [[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]],
 
          [[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False

In [50]:
'fix' in "fix@0.95-mix"

True

In [237]:
dir(dl)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [14]:
from datasets import VOCSegmentation

/usr/lib/python3/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/lib/python3/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorc

In [15]:
dataset = VOCSegmentation()

In [258]:
cd ..

/home/ubuntu/SemiFL_extension/src


In [266]:
import sys
import numpy as np
sys.getsizeof(dataset.data)

184

In [267]:
np.shape(dataset.data)

(2913, 3, 256, 256)

In [268]:
data_copy = dataset.data.copy()

In [20]:
dataset.data._mmap

<mmap.mmap closed=False, access=ACCESS_READ, length=2290876544, pos=0, offset=0>

In [21]:
dc._mmap

<mmap.mmap closed=False, access=ACCESS_READ, length=2290876544, pos=0, offset=0>

In [270]:
data_copy._mmap

In [16]:
dc = dataset.data

In [22]:
np.info(dc)

class:  memmap
shape:  (2913, 3, 256, 256)
strides:  (786432, 262144, 1024, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f6ee7600080
byteorder:  little
byteswap:  False
type: float32


In [23]:
np.info(dataset.data)

class:  memmap
shape:  (2913, 3, 256, 256)
strides:  (786432, 262144, 1024, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f6ee7600080
byteorder:  little
byteswap:  False
type: float32


In [278]:
np.info(data_copy)

class:  memmap
shape:  (2913, 3, 256, 256)
strides:  (786432, 262144, 1024, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f1007b3f010
byteorder:  little
byteswap:  False
type: float32


In [27]:
np.random.randint(0,21,(513,))[0]

15

In [33]:
fdata[1,:].shape

torch.Size([3, 256, 256])

In [36]:
import torchvision
gaus = torchvision.transforms.v2.GaussianBlur(kernel_size=(5,5), sigma=(0.1))

In [39]:
gaus(fdata[1,:]).shape

torch.Size([3, 256, 256])

In [40]:
fdata[1,:].shape

torch.Size([3, 256, 256])

In [ ]:
index = torch.randint(0, len(self.dataset), (1,)).item()


In [43]:
ds1,ds2 = dataset

ValueError: too many values to unpack (expected 2)

In [63]:
torch.cat((fdata[1,:],fdata[1,:]))[0,...].unsqueeze(0).shape

torch.Size([1, 256, 256])

In [64]:
def compute_pixel_accuracy(pred_mask,true_mask):
    pred_mask = torch.argmax(pred_mask,1)
    correct_pixels = torch.sum(true_mask == pred_mask)
    total_pixels = true_mask.view(-1).size()[0]
    pixel_accuracy = correct_pixels / total_pixels
    return pixel_accuracy.item()

In [69]:
ip = torch.randint(0,21,(12,256,256))
pred = torch.rand(12,21,256,256)
ipp = torch.argmax(pred,1)

In [70]:
compute_pixel_accuracy(pred,ipp)

1.0

In [ ]:
!python train_classifier_ssfl.py --data_name voc --model_name deeplab --control_name 500_fix@0.95-mix_3_0.5_iid_5-5_0.5_0_1

In [ ]:
git clone https://github.com/Karthik2924/SemiFL_extension.git

In [3]:
mkdir data

In [4]:
cd data

/home/ubuntu/SemiFL_extension/src/data


In [6]:
!gdown 1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65

Downloading...
From (original): https://drive.google.com/uc?id=1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65
From (redirected): https://drive.google.com/uc?id=1vXyVtH38tXe6qgtB1F4xgOqg3PuAtU65&confirm=t&uuid=952b0318-c80b-4622-8d8d-5bd4d1153b50
To: /home/ubuntu/SemiFL_extension/src/data/images.npy
100%|███████████████████████████████████████| 2.29G/2.29G [00:14<00:00, 154MB/s]


In [7]:
!gdown 1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ

Downloading...
From (original): https://drive.google.com/uc?id=1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ
From (redirected): https://drive.google.com/uc?id=1MH7vQJJj_UxsCrpcNzZZvs-U-NaefsGQ&confirm=t&uuid=4c3f81bc-da88-4f68-97e0-0dbb8c172905
To: /home/ubuntu/SemiFL_extension/src/data/masks.npy
100%|███████████████████████████████████████| 1.53G/1.53G [00:09<00:00, 165MB/s]


In [2]:
pip install gdown torchmetrics anytree

[Errno 2] No such file or directory: 'data'
/home/ubuntu/SemiFL_extension/src


In [18]:
from itertools import compress
import torch.nn as nn
import torch.nn.functional as F
import torch
import numpy as np

In [19]:
import torch
from torch.utils.data import Dataset

class SimpleDataset(Dataset):
    def __init__(self):
        #self.num_samples = num_samples
        self.data = torch.rand(100,3,128,128)
        self.target = torch.rand(100,1,128,128)
        self.transform = None
        self.ind = np.arange(100)


    def __len__(self):
        return len(self.data)
        #return self.num_samples

    def __getitem__(self, idx):
        didx = self.ind[idx]
        if self.transform:
            return {'data':self.transform(torch.tensor(self.data[didx])), 'target':self.transform(self.target[idx])}
        else:
            return {'data': torch.tensor(self.data[didx]), 'target':self.target[idx]}

# Create an instance of the ToyDataset
toy_dataset = SimpleDataset()


In [ ]:
 mix_data : torch.Size([32, 3, 256, 256])
 mix_target : torch.Size([32, 256, 256, 2])

In [55]:
mt = torch.rand(32,256,256,2)
mt[...,0].shape

torch.Size([32, 256, 256])

In [225]:
toy_dataset[9]

{'data': tensor([[[0.1558, 0.5554, 0.7492,  ..., 0.7182, 0.6796, 0.6616],
          [0.0715, 0.1611, 0.1820,  ..., 0.7437, 0.8855, 0.1173],
          [0.9753, 0.6165, 0.5458,  ..., 0.2451, 0.8714, 0.9966],
          ...,
          [0.7623, 0.5801, 0.5785,  ..., 0.9306, 0.1995, 0.6560],
          [0.3178, 0.5615, 0.6404,  ..., 0.4690, 0.6695, 0.1623],
          [0.7191, 0.6891, 0.3419,  ..., 0.6456, 0.5513, 0.0148]],
 
         [[0.7347, 0.0358, 0.0323,  ..., 0.3718, 0.2759, 0.4531],
          [0.5653, 0.4994, 0.9425,  ..., 0.9436, 0.1095, 0.9461],
          [0.7066, 0.5367, 0.4919,  ..., 0.4237, 0.4636, 0.9858],
          ...,
          [0.5698, 0.4981, 0.6093,  ..., 0.7908, 0.8211, 0.4857],
          [0.8211, 0.7413, 0.0222,  ..., 0.0151, 0.6716, 0.4168],
          [0.1369, 0.1983, 0.8129,  ..., 0.9479, 0.2614, 0.0351]],
 
         [[0.6801, 0.1116, 0.7881,  ..., 0.2024, 0.5684, 0.4633],
          [0.6290, 0.7158, 0.5946,  ..., 0.4991, 0.0343, 0.2133],
          [0.4969, 0.4387, 0.126

In [ ]:
output_, input_ = {}, {}
output_['target'] = torch.cat(output, dim=0)
input_['target'] = torch.cat(target, dim=0)
#output_['target'] = F.softmax(output_['target'], dim=-1)
output_['target'] = F.softmax(output_['target'], dim=1)

In [3]:
op = torch.rand(12, 21, 256, 256)
ip = torch.randint(0,21,(12,256,256))
print(op.shape,ip.shape)

torch.Size([12, 21, 256, 256]) torch.Size([12, 256, 256])


In [4]:
op = F.softmax(op, dim=1)

In [5]:
op.shape

torch.Size([12, 21, 256, 256])

In [6]:
def make_hard_pseudo_label( soft_pseudo_label):
    #max_p, hard_pseudo_label = torch.max(soft_pseudo_label, dim=-1)
    max_p, hard_pseudo_label = torch.max(soft_pseudo_label, dim=1)
    print(max_p.shape)
    mv = max_p.view(max_p.shape[0],-1).sum(1)
    a,b,c = max_p.shape
    total = b*c
    #mask = max_p.ge(cfg['threshold'])
    #mask = max_p.ge(0.08)
    mask = mv.ge(total*0.07225)
    return hard_pseudo_label, mask


In [7]:
nlab,mask = make_hard_pseudo_label(op)

torch.Size([12, 256, 256])


In [8]:
torch.unique(mask,return_counts = True)

(tensor([False,  True]), tensor([7, 5]))

In [210]:
sup = list(compress(nlab, mask))

In [226]:
nlab.shape

torch.Size([12, 256, 256])

In [9]:
fdata = torch.rand(25,3,256,256)

In [10]:
ind = torch.arange(12)

In [11]:
fdata[ind,:][mask].shape

torch.Size([5, 3, 256, 256])

In [12]:
r = torch.rand(12,256,256)
r.view(12,-1).sum(1).shape

torch.Size([12])

In [246]:
mask

tensor([False, False, False,  True, False,  True,  True, False, False, False,
        False,  True])

In [228]:
torch.stack(sup).shape

torch.Size([4, 256, 256])

In [45]:
sup1 = list(compress(ip, mask))

In [48]:
type(torch.stack(sup1)) == torch.Tensor

True

In [47]:
type(sup1)

list

In [231]:
ip.shape

torch.Size([12, 256, 256])

In [13]:
from torch.utils.data import DataLoader, Dataset
dl = DataLoader(toy_dataset,batch_size = 16,shuffle = False)

In [239]:
batch = next(iter(dl))

In [241]:
b1 = next(iter(dl))

In [243]:
b2 = next(iter(dl))

In [245]:
b1['data'] == batch['data'], b2['data'] == batch['data']

(tensor([[[[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]],
 
          [[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False]],
 
          [[False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           [False, False, False,  ..., False, False, False],
           ...,
           [False, False, False

In [50]:
'fix' in "fix@0.95-mix"

True

In [237]:
dir(dl)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_is_protocol',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [14]:
from datasets import VOCSegmentation

/usr/lib/python3/dist-packages/torchvision/datapoints/__init__.py:12: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorch/vision/issues/6753, and you can also check out https://github.com/pytorch/vision/issues/7319 to learn more about the APIs that we suspect might involve future changes. You can silence this warning by calling torchvision.disable_beta_transforms_warning().
  warnings.warn(_BETA_TRANSFORMS_WARNING)
/usr/lib/python3/dist-packages/torchvision/transforms/v2/__init__.py:54: UserWarning: The torchvision.datapoints and torchvision.transforms.v2 namespaces are still Beta. While we do not expect major breaking changes, some APIs may still change according to user feedback. Please submit any feedback you may have in this issue: https://github.com/pytorc

In [15]:
dataset = VOCSegmentation()

In [258]:
cd ..

/home/ubuntu/SemiFL_extension/src


In [266]:
import sys
import numpy as np
sys.getsizeof(dataset.data)

184

In [267]:
np.shape(dataset.data)

(2913, 3, 256, 256)

In [268]:
data_copy = dataset.data.copy()

In [20]:
dataset.data._mmap

<mmap.mmap closed=False, access=ACCESS_READ, length=2290876544, pos=0, offset=0>

In [21]:
dc._mmap

<mmap.mmap closed=False, access=ACCESS_READ, length=2290876544, pos=0, offset=0>

In [270]:
data_copy._mmap

In [16]:
dc = dataset.data

In [22]:
np.info(dc)

class:  memmap
shape:  (2913, 3, 256, 256)
strides:  (786432, 262144, 1024, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f6ee7600080
byteorder:  little
byteswap:  False
type: float32


In [23]:
np.info(dataset.data)

class:  memmap
shape:  (2913, 3, 256, 256)
strides:  (786432, 262144, 1024, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f6ee7600080
byteorder:  little
byteswap:  False
type: float32


In [278]:
np.info(data_copy)

class:  memmap
shape:  (2913, 3, 256, 256)
strides:  (786432, 262144, 1024, 4)
itemsize:  4
aligned:  True
contiguous:  True
fortran:  False
data pointer: 0x7f1007b3f010
byteorder:  little
byteswap:  False
type: float32


In [27]:
np.random.randint(0,21,(513,))[0]

15

In [33]:
fdata[1,:].shape

torch.Size([3, 256, 256])

In [36]:
import torchvision
gaus = torchvision.transforms.v2.GaussianBlur(kernel_size=(5,5), sigma=(0.1))

In [39]:
gaus(fdata[1,:]).shape

torch.Size([3, 256, 256])

In [40]:
fdata[1,:].shape

torch.Size([3, 256, 256])

In [ ]:
index = torch.randint(0, len(self.dataset), (1,)).item()


In [43]:
ds1,ds2 = dataset

ValueError: too many values to unpack (expected 2)

In [63]:
torch.cat((fdata[1,:],fdata[1,:]))[0,...].unsqueeze(0).shape

torch.Size([1, 256, 256])

In [64]:
def compute_pixel_accuracy(pred_mask,true_mask):
    pred_mask = torch.argmax(pred_mask,1)
    correct_pixels = torch.sum(true_mask == pred_mask)
    total_pixels = true_mask.view(-1).size()[0]
    pixel_accuracy = correct_pixels / total_pixels
    return pixel_accuracy.item()

In [69]:
ip = torch.randint(0,21,(12,256,256))
pred = torch.rand(12,21,256,256)
ipp = torch.argmax(pred,1)

In [70]:
compute_pixel_accuracy(pred,ipp)

1.0